# Train on US region

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import math
from copy import deepcopy
import numpy as np

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open('./data/England/queries_filtered.txt') as f:
    raw_us_queries = f.read().splitlines()
query_ids_list = [int(r.split('\t')[0]) for r in raw_us_queries]
query_ids = set(query_ids_list)
q_freq_us_raw = pd.read_csv('./data/US/Q_freq.csv')

In [ ]:
query_ids_dict = {int(q): k for q, k in [query.split('\t') for query in raw_us_queries]}

In [ ]:
q_freq_raw_filtered_us = q_freq_us_raw[q_freq_us_raw['Query'].isin(query_ids)]
q_freq_raw_filtered_us = q_freq_raw_filtered_us.pivot(index='Day', columns='Query', values='Frequency')
q_freq_raw_filtered_us = q_freq_raw_filtered_us.reindex(range(q_freq_raw_filtered_us.index.min(), q_freq_raw_filtered_us.index.max() + 1), fill_value=None)
q_freq_raw_filtered_us = q_freq_raw_filtered_us.fillna(0)

In [ ]:
from datetime import datetime

def days_between(d1, d2, time_format="%Y-%m-%d"):
    d1 = datetime.strptime(d1, time_format)
    d2 = datetime.strptime(d2, time_format)
    return (d2 - d1).days

def day_index(d, time_format="%Y-%m-%d"):
    start_date = '2004-01-01'
    # Reformat the start date to match the time format
    start_date = datetime.strptime(start_date, "%Y-%m-%d").strftime(time_format)
    return days_between(start_date, d, time_format=time_format)

def process_query_data(q_freq_raw, query_ids=query_ids):
    q_freq_raw_filtered  = q_freq_raw[q_freq_raw['Query'].isin(query_ids)]
    q_freq_raw_filtered = q_freq_raw_filtered.pivot(index='Day', columns='Query', values='Frequency')

    # Fill in missing days with NaN
    q_freq_raw_filtered = q_freq_raw_filtered.reindex(range(q_freq_raw_filtered.index.min(), q_freq_raw_filtered.index.max() + 1), fill_value=None)
    # Replace missing values with zeroes
    q_freq_raw_filtered = q_freq_raw_filtered.fillna(0)

    # Apply harmonic mean to the data: for each day, use the harmonic weighted mean of the frequencies back to T-6 (or the start of the data if there are fewer than 14 days of data before T) to smooth the data
    # harmonic_length = 7
    # harmonic_filter = np.array([(harmonic_length-i)/harmonic_length for i in range(harmonic_length+1)])
    # harmonic_filter = np.array([1/i for i in range(1, harmonic_length+1)])
    # harmonic_weight_factor = [1 / sum(harmonic_filter[:i]) for i in range(1, harmonic_length+1)]

    harmonic_length = 14
    harmonic_filter = np.array([1/(i+1) for i in range(harmonic_length+1)])
    # harmonic_filter = np.array([1/i for i in range(1, harmonic_length+1)])
    harmonic_weight_factor = [1 / sum(harmonic_filter[:i]) for i in range(1, harmonic_length+1)]

    q_freq_raw_filtered_smoothed = deepcopy(q_freq_raw_filtered)
    for i in range(1, q_freq_raw_filtered.shape[0]):
        harmonic_range = min(i + 1, harmonic_length)
        harmonic_values = q_freq_raw_filtered.iloc[i - harmonic_range + 1:i + 1].values
        harmonic_weights = harmonic_filter[:harmonic_range][::-1]
        weighted_sum = (harmonic_values * harmonic_weights[:, None]).sum(axis=0)
        q_freq_raw_filtered_smoothed.iloc[i] = weighted_sum * harmonic_weight_factor[harmonic_range - 1]

    # Min-max normalize the data, grouping by query
    query_min = q_freq_raw_filtered_smoothed.min()
    query_max = q_freq_raw_filtered_smoothed.max()
    q_freq_raw_filtered_smoothed = (q_freq_raw_filtered_smoothed - query_min) / (query_max - query_min)
    
    # Fill NaN values with 0
    q_freq_raw_filtered_smoothed = q_freq_raw_filtered_smoothed.fillna(0)

    return q_freq_raw_filtered_smoothed, query_min, query_max

q_freq_us_smoothed, query_min_us, query_max_us = process_query_data(q_freq_us_raw)

In [ ]:
def query_data_for_season(start_date, end_date, q_freq_raw_filtered):
    start_day = day_index(start_date)
    end_day = day_index(end_date)
    q_freq_raw_filtered_season = q_freq_raw_filtered.iloc[start_day:end_day+1]
    return q_freq_raw_filtered_season

all_seasons = [('2008-09-01', '2009-08-31'), ('2009-09-01', '2010-08-31'), ('2010-09-01', '2011-08-31'), ('2011-09-01', '2012-08-31'), ('2012-09-01', '2013-08-31'), ('2013-09-01', '2014-08-31'), ('2014-09-01', '2015-08-31'), ('2015-09-01', '2016-08-31'), ('2016-09-01', '2017-08-31'), ('2017-09-01', '2018-08-31'), ('2018-09-01', '2019-08-31')]

test_seasons = all_seasons[-2:]
train_seasons = all_seasons[:-2]
training_query_data = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons]
testing_query_data = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons]

In [ ]:
ili_raw_us = pd.read_csv('./data/US/ILINet.csv', skiprows=1)

ili_raw_us = ili_raw_us[['YEAR', 'WEEK', 'ILITOTAL']]

# Turn year and week number into date
def year_and_week_to_date(year, week):
    # For example, year 2008 and week 35 (end of week) is 2008

    # Find the first day of the year
    first_day = datetime.strptime(f'{year}-01-01', "%Y-%m-%d")
    # Find the Wednesday of the week
    first_day_of_week = first_day + pd.DateOffset(weeks=week-1, days=2-first_day.weekday())
    
    # Return a string
    return first_day_of_week.strftime("%Y-%m-%d")

ili_raw_us['WeekStart'] = ili_raw_us.apply(lambda x: year_and_week_to_date(x['YEAR'], x['WEEK']), axis=1)
ili_raw_us

# From 2008 to 2019
us_population_dict = {2008: 305694910, 2009: 308512035, 2010: 311182845, 2011: 313876608, 2012: 316651321, 2013: 319375166, 2014: 322033964, 2015: 324607776, 2016: 327210198, 2017: 329791231, 2018: 332140037, 2019: 334319671}

ili_raw_us['Population'] = np.nan
for year, population in us_population_dict.items():
    first_date_index = ili_raw_us[ili_raw_us['YEAR'] == year].index[0]
    ili_raw_us.at[first_date_index, 'Population'] = population

# Linear interpolation for missing values
ili_raw_us['ILITOTAL'] = ili_raw_us['ILITOTAL'].interpolate(method='linear')
ili_raw_us['Population'] = ili_raw_us['Population'].interpolate(method='linear')

# Calculate ILI rate
ili_raw_us['ILIRate'] = ili_raw_us['ILITOTAL'] / ili_raw_us['Population'] * 100000

dataset_start_day = day_index('2008-09-01')
dataset_end_day = day_index('2019-08-31')

def preprocess_ili_data(ili_raw, time_format="%Y-%m-%d", training_end_date='2017-08-31', training_start_date='2008-09-01'):
    ili_raw['Day'] = ili_raw['WeekStart'].apply(day_index, time_format=time_format)
    ili_raw = ili_raw.set_index('Day')
    ili_raw = ili_raw[~ili_raw.index.duplicated(keep='first')]
    ili_raw = ili_raw.reindex(range(ili_raw.index.min(), ili_raw.index.max() + 1), fill_value=None)
    ili_raw = ili_raw.reset_index()

    ili_raw = ili_raw[['Day', 'ILIRate']]
    ili_raw = ili_raw[ili_raw['Day'].between(dataset_start_day, dataset_end_day)]

    # Interpolate missing values with linear interpolation
    ili_raw['ILIRate'] = ili_raw['ILIRate'].interpolate(method='linear')

    # Extrapolate so starting days and ending days have values
    ili_raw['ILIRate'] = ili_raw['ILIRate'].bfill()

    ili_raw_training = ili_raw[ili_raw['Day'].between(day_index(training_start_date), day_index(training_end_date))]
    # Find max and min for training data
    max_ili = ili_raw_training['ILIRate'].max()
    min_ili = ili_raw_training['ILIRate'].min()

    # Normalize ILI rates
    ili_raw['ILIRate'] = (ili_raw['ILIRate'] - min_ili) / (max_ili - min_ili)
    
    return ili_raw, min_ili, max_ili

def ili_data_for_season(start_date, end_date, ili_raw):
    start_day = day_index(start_date)
    end_day = day_index(end_date)
    ili_raw_season = ili_raw[ili_raw['Day'].between(start_day, end_day)]

    # Set Day as index
    ili_raw_season = ili_raw_season.set_index('Day')

    return ili_raw_season

In [ ]:
processed_ili_data, min_ili, max_ili = preprocess_ili_data(ili_raw_us)

training_ili_data = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons]
testing_ili_data = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons]

In [ ]:
# Plot a random query in the dataset
import matplotlib.pyplot as plt

query_id = 1
unsmoothed_time_series = q_freq_raw_filtered_us[query_id][day_index('2015-09-01'):day_index('2015-10-31') + 1]
average_smoothed_time_series = unsmoothed_time_series.rolling(window=7).mean()
harmonic_smoothed_time_series = deepcopy(unsmoothed_time_series)

harmonic_length = 7
harmonic_filter = np.array([(harmonic_length-i)/harmonic_length for i in range(harmonic_length+1)])
harmonic_weight_factor = [1 / sum(harmonic_filter[:i]) for i in range(1, harmonic_length+1)]

for i in range(1, len(harmonic_smoothed_time_series)):
    harmonic_range = min(i + 1, harmonic_length)
    harmonic_values = unsmoothed_time_series[i - harmonic_range + 1:i + 1]
    harmonic_weights = harmonic_filter[:harmonic_range][::-1]
    weighted_sum = (harmonic_values * harmonic_weights).sum()
    harmonic_smoothed_time_series[day_index('2015-09-01') + i] = weighted_sum * harmonic_weight_factor[harmonic_range - 1]

# Limit x axis to 2015-09-01 to 2015-10-31
plt.plot(unsmoothed_time_series, label="Unsmoothed")
plt.plot(average_smoothed_time_series, label="Average smoothed")
plt.plot(harmonic_smoothed_time_series, label="Adjusted smoothed")

plt.title(f"Query 'flu medicine' between 2015-09-01 and 2015-10-31")

plt.legend()

plt.tight_layout()
plt.savefig('plots/query_smoothing.pdf')

In [ ]:
# Plot ILI data by seasons, combined in one plot

plt.plot(pd.concat(training_ili_data), label="Training")
plt.plot(pd.concat(testing_ili_data), label="Testing")

plt.xlabel("Day")
plt.ylabel("Normalized ILI Rate")

# Change the x-axis to show the dates of start of each season
plt.xticks([day_index(start_date) for start_date, _ in train_seasons + test_seasons], [start_date for start_date, _ in train_seasons + test_seasons], rotation=60)

plt.legend()
plt.tight_layout()
plt.savefig('ILI_Rate_by_Season.pdf', bbox_inches='tight')
plt.show()

In [ ]:
def pearson_correlation(query_id, query_data, ili_data):
    query_values = query_data[query_id].values
    ili_values = ili_data['ILIRate'].values
    return pd.Series(query_values).corr(pd.Series(ili_values))

def filter_correlated_queries(query_data_seasons, ili_data_seasons, num_of_features=50):
    qids_correlated = set()
    qids_candidates = set(query_data_seasons[0].columns)
    qid_correlation_map = {}
    for qid in qids_candidates:
        qid_not_in_all_data = False
        qid_correlation = 0
        # Check not all values are constant
        # Only calculate correlation for the last 5 seasons
        for i, query_data in enumerate(query_data_seasons[-5:]):
            if qid not in query_data.columns:
                qid_not_in_all_data = True
                break
            if query_data[qid].std() == 0:
                qid_correlation += 0
                continue
            qid_correlation += pearson_correlation(qid, query_data, ili_data_seasons[i])

        if qid_not_in_all_data:
            continue
        qid_correlation_map[qid] = qid_correlation

    qid_correlation_map = {k: v for k, v in sorted(qid_correlation_map.items(), key=lambda item: item[1], reverse=True)}
    qids_all_sorted = list(qid_correlation_map.keys())
    qids_correlated = list(qid_correlation_map.keys())[:num_of_features]
    
    query_data_seasons_correlated = [query_data[qids_correlated] for query_data in query_data_seasons]
    return query_data_seasons_correlated, qids_correlated, qids_all_sorted

Qtrain_correlated, qids_correlated, qids_all_sorted = filter_correlated_queries(training_query_data, training_ili_data, num_of_features=50)
Qtest_correlated = [query_data[list(qids_correlated)] for query_data in testing_query_data]

In [ ]:
# Nowcast Neural Network
import torch
import torch.nn as nn
import torch.optim as optim

# Fully Connected Neural Network. Input size is the number of queries * window size, output size is 1
class NowcastFCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NowcastFCNN, self).__init__()
        self.hidden_size = hidden_size
        self.activation = nn.ReLU()
        # self.bn1 = nn.BatchNorm1d(hidden_size)
        # self.bn2 = nn.BatchNorm1d(hidden_size // 2)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        # out = self.bn1(out)
        out = self.activation(out)

        out = self.fc2(out)
        out = self.activation(out)
        # out = self.bn2(out)

        out = self.fc3(out)
        return out

In [ ]:
# Code for training and testing FCNN

# Expand Y from day T to from T-output_window_size//2 to T+output_window_size//2 for each day T. output_window_size needs to be odd
def extrapolate_y_windows(y_tensor, output_window_size):
    expanded_y = []
    half_window = output_window_size // 2
    for i in range(len(y_tensor)):
        start = max(0, i - half_window)
        end = min(len(y_tensor), i + half_window + 1)
        window = y_tensor[start:end].flatten()
        if len(window) < output_window_size:
            if start == 0:  # extrapolate at the beginning
                extrapolated_values = torch.linspace(window[0], window[0], half_window - i)
                window = torch.cat((extrapolated_values, window))
            if end == len(y_tensor):  # extrapolate at the end
                extrapolated_values = torch.linspace(window[-1], window[-1], half_window - len(y_tensor) + i + 1)
                window = torch.cat((window, extrapolated_values))
        expanded_y.append(window)
    return torch.stack(expanded_y)

# Preprocess a split
def preprocess_fold(X, Y, input_window_size=7, output_window_size=7):
    X_windows = []
    Y_windows = []
    for fold in range(len(X)):
        X_windows.append([])
        Y_windows.append([])
        for i in range(input_window_size, len(X[fold])):
            X_windows[fold].append(X[fold][i-input_window_size:i].values.flatten())
            Y_windows[fold].append(Y[fold].iloc[i]['ILIRate'])
        X_windows[fold] = torch.tensor(X_windows[fold], dtype=torch.float32)
        Y_windows[fold] = torch.tensor(Y_windows[fold], dtype=torch.float32)

    # Combine training data into one tensor
    X_windows = torch.cat(X_windows)
    Y_windows = torch.cat(Y_windows).view(-1, 1)
    Y_expanded = extrapolate_y_windows(Y_windows, output_window_size)

    return X_windows, Y_expanded

def validation_fold(X_train, Y_train):
    # For validation, use 0-100 days from season -3, 101-200 days from season -2, 201-end from season -1, concatenate them
    # Concatenate the data from the three seasons
    X_val = [X_train[-3][:100], X_train[-2][100:200], X_train[-1][200:]]
    Y_val = [Y_train[-3][:100], Y_train[-2][100:200], Y_train[-1][200:]]

    # Remove the validation data (only these days, not the entire seasons) from training data
    X_train[-3] = X_train[-3][100:]
    X_train[-2] = pd.concat([X_train[-2][:100], X_train[-2][200:]])
    X_train[-1] = X_train[-1][:200]
    Y_train[-3] = Y_train[-3][100:]
    Y_train[-2] = pd.concat([Y_train[-2][:100], Y_train[-2][200:]])
    Y_train[-1] = Y_train[-1][:200]

    return X_val, Y_val, X_train, Y_train

def test_model(model, X_train_windows, Y_train_windows, X_test_windows, Y_test_windows, output_size=7, plot_graph=True, min_ili=None, max_ili=None):
        # Test the model. Only pick the T day for testing
        model.eval()
        Y_pred = model(X_test_windows)
        Y_pred = Y_pred[:, output_size // 2].view(-1, 1)
        Y_test_windows = Y_test_windows[:, output_size // 2].view(-1, 1)

        Y_pred_train = model(X_train_windows)
        Y_pred_train = Y_pred_train[:, output_size // 2].view(-1, 1)
        Y_train_windows = Y_train_windows[:, output_size // 2].view(-1, 1)

        # Denormalize the data
        if min_ili is not None and max_ili is not None:
            Y_pred = Y_pred * (max_ili - min_ili) + min_ili
            Y_test_windows = Y_test_windows * (max_ili - min_ili) + min_ili
            Y_pred_train = Y_pred_train * (max_ili - min_ili) + min_ili
            Y_train_windows = Y_train_windows * (max_ili - min_ili) + min_ili

        # Plot the results
        if plot_graph:
            import matplotlib.pyplot as plt
            plt.plot(Y_test_windows.cpu().numpy(), label='True')
            plt.plot(Y_pred.cpu().detach().numpy(), label='Predicted')
            plt.legend()
            plt.show()

            # Also test the model on the training data
            plt.plot(Y_train_windows.cpu().numpy(), label='True')
            plt.plot(Y_pred_train.cpu().detach().numpy(), label='Predicted')
            plt.legend()
            plt.show()

        # MSE as evaluation
        metric = nn.L1Loss()
        train_loss = metric(Y_pred_train, Y_train_windows).item()
        test_loss = metric(Y_pred, Y_test_windows).item()

        # Forecast skill as evaluation
        mse_metric = nn.MSELoss()
        forecast_skill = 1 - mse_metric(Y_pred_train, Y_train_windows) / mse_metric(Y_pred, Y_test_windows)

        # Pearson Correlation as evaluation
        correlation = Y_pred.cpu().detach().numpy().flatten()
        true_values = Y_test_windows.cpu().numpy().flatten()
        correlation = pd.Series(correlation).corr(pd.Series(true_values))
        return train_loss, test_loss, correlation, forecast_skill

def prepare_test_data(model, X_test_windows, Y_test_windows, output_size=7, min_ili=None, max_ili=None):
    Y_pred = model(X_test_windows)
    Y_pred = Y_pred[:, output_size // 2].view(-1, 1)
    Y_test_windows = Y_test_windows[:, output_size // 2].view(-1, 1)

    # Denormalize the data
    if min_ili is not None and max_ili is not None:
        Y_pred = Y_pred * (max_ili - min_ili) + min_ili
        Y_test_windows = Y_test_windows * (max_ili - min_ili) + min_ili

    return Y_pred, Y_test_windows

def prepare_test_data_scaled(model, X_test_windows, Y_test_windows, output_size=7, min_ili=None, max_ili=None, min_ili_scaled=None, max_ili_scaled=None):
    Y_pred = model(X_test_windows)
    Y_pred = Y_pred[:, output_size // 2].view(-1, 1)
    Y_test_windows = Y_test_windows[:, output_size // 2].view(-1, 1)

    # Denormalize the data
    if min_ili is not None and max_ili is not None:
        Y_pred = Y_pred * (max_ili - min_ili) + min_ili

    if min_ili_scaled is not None and max_ili_scaled is not None:
        Y_test_windows = Y_test_windows * (max_ili_scaled - min_ili_scaled) + min_ili_scaled

    return Y_pred, Y_test_windows

In [ ]:
def prepare_data(Qtrain, Qtest, ILITrain, ILITest, input_window_size=7, output_window_size=7, num_of_features=50, batch_size=28, random_seed=0):
    # Determine the queries that are correlated with ILI rates based on training data, then filter the testing data based on the correlated queries
    Qtrain_correlated, qids_correlated, _ = filter_correlated_queries(Qtrain, ILITrain, num_of_features=num_of_features)
    Qtest_correlated = [query_data[list(qids_correlated)] for query_data in Qtest]

    X_train = deepcopy(Qtrain_correlated)
    Y_train = deepcopy(ILITrain)
    X_test = Qtest_correlated
    Y_test = ILITest

    X_val, Y_val, X_train, Y_train = validation_fold(X_train, Y_train)

    X_train, Y_train = preprocess_fold(X_train, Y_train, input_window_size=input_window_size, output_window_size=output_window_size)
    X_val, Y_val = preprocess_fold(X_val, Y_val, input_window_size=input_window_size, output_window_size=output_window_size)
    X_test, Y_test = preprocess_fold(X_test, Y_test, input_window_size=input_window_size, output_window_size=output_window_size)

    # Move data to device
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)
    X_val = X_val.to(device)
    Y_val = Y_val.to(device)

    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        random.seed(worker_seed)

    g = torch.Generator()
    g.manual_seed(random_seed)

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)

    return train_loader, X_train, Y_train, X_test, Y_test, X_val, Y_val

def prepare_data_without_validation(Qtrain, Qtest, ILITrain, ILITest, input_window_size=7, output_window_size=7, num_of_features=50, batch_size=28):
    # Determine the queries that are correlated with ILI rates based on training data, then filter the testing data based on the correlated queries
    Qtrain_correlated, qids_correlated, _ = filter_correlated_queries(Qtrain, ILITrain, num_of_features=num_of_features)
    Qtest_correlated = [query_data[list(qids_correlated)] for query_data in Qtest]

    X_train = deepcopy(Qtrain_correlated)
    Y_train = deepcopy(ILITrain)
    X_test = Qtest_correlated
    Y_test = ILITest

    X_train, Y_train = preprocess_fold(X_train, Y_train, input_window_size=input_window_size, output_window_size=output_window_size)
    X_test, Y_test = preprocess_fold(X_test, Y_test, input_window_size=input_window_size, output_window_size=output_window_size)

    # Move data to device
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, X_train, Y_train, X_test, Y_test

In [ ]:
def train_nn(train_loader, X_train, Y_train, X_val, Y_val, hidden_size=128, learning_rate=0.01, max_epochs=200, input_window_size=7, output_window_size=7, plot_graph=False):
    input_size = X_train.shape[1]
    output_size = output_window_size

    print("Input size: ", input_size)

    model = NowcastFCNN(input_size, hidden_size, output_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    patience = 10
    patience_counter = 0
    best_model = None
    best_val_loss = math.inf
    loss_fn = nn.MSELoss()

    warmup_epochs = 10
    val_losses = []
        
    # Train the model
    for i in tqdm(range(max_epochs)):
        # Load data in batches
        for X_batch, Y_batch in train_loader:
            model.train()
            optimizer.zero_grad()
            if(X_batch.shape[0] == 1):
                continue
            Y_pred = model(X_batch)
            loss = loss_fn(Y_pred, Y_batch) + 1e-3 * sum(torch.linalg.norm(p, 1) for p in model.parameters())
            loss.backward()
            optimizer.step()
        _, val_loss, _, _ = test_model(model, X_train, Y_train, X_val, Y_val, output_size=output_size, plot_graph=False)
        val_losses.append(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model)
            patience_counter = 0
        elif i > warmup_epochs:
            patience_counter += 1
        if patience_counter >= patience and i > warmup_epochs:
            break

    if plot_graph:
        plt.plot(val_losses)
        plt.show()

    return best_model, best_val_loss

def plot_predictions(model, X_test, Y_test, start_date, end_date, output_size=7, min_ili=None, max_ili=None,  y_definition='ILI Rate', filename="nowcast.pdf", savefig=False):
    Y_pred, Y_test_post = prepare_test_data(model, X_test, Y_test, output_size=output_size, min_ili=min_ili, max_ili=max_ili)

    # Find the day where difference is largest
    diff = Y_pred - Y_test_post
    max_diff = diff.abs().max()
    max_diff_day = diff.abs().argmax().item()
    max_diff_date = pd.Timedelta(days=max_diff_day) + pd.to_datetime(start_date)
    print(f"Max difference: {max_diff.item():.2f}% on {max_diff_date:%m/%d}")

    # Calculate the correlation between the predicted and true values
    pred_values = Y_pred.cpu().detach().numpy().flatten()
    true_values = Y_test_post.cpu().numpy().flatten()
    correlation = pd.Series(pred_values).corr(pd.Series(true_values))
    print(f"Correlation: {correlation*100:.2f}%")

    # Calculate the mean absolute error
    mae = nn.L1Loss()
    mae_loss = mae(Y_pred, Y_test_post).item()
    print(f"MAE: {mae_loss:.3f}%")

    # Plot the results
    import matplotlib.pyplot as plt
    plt.plot(Y_test_post.cpu().numpy(), label='True')
    plt.plot(Y_pred.cpu().detach().numpy(), label='Predicted')

    # Label the x-axis with the dates
    days = pd.date_range(start=start_date, end=end_date, freq='D')
    days = [day.strftime("%m-%d") for day in days]
    plt.xticks(range(0, len(days), 30), days[::30], rotation=60)
    plt.ylim(0, 30)

    # Label the y-axis with definition
    plt.ylabel(y_definition)
    plt.xlabel('Date')

    plt.legend()
    
    # Save the figure as a pdf
    if savefig:
        plt.savefig(filename, format='pdf')
    plt.show()

def plot_predictions_scaled(model, X_test, Y_test, start_date, end_date, output_size=7, min_ili=None, max_ili=None, min_ili_target=None, max_ili_target=None, y_definition='ILI Rate', max_query=None, verbal=True):
    Y_pred, Y_test_post = prepare_test_data_scaled(model, X_test, Y_test, output_size=output_size, min_ili=min_ili, max_ili=max_ili, min_ili_scaled=min_ili_target, max_ili_scaled=max_ili_target)

    if max_query is not None:
        Y_pred = Y_pred * max_query

    # Find the day where difference is largest
    diff = Y_pred - Y_test_post
    max_diff = diff.abs().max()
    max_diff_day = diff.abs().argmax().item()
    max_diff_date = pd.Timedelta(days=max_diff_day) + pd.to_datetime(start_date)
    
    # Calculate the correlation between the predicted and true values
    pred_values = Y_pred.cpu().detach().numpy().flatten()
    true_values = Y_test_post.cpu().numpy().flatten()
    correlation = pd.Series(pred_values).corr(pd.Series(true_values))
    
    # Calculate the mean absolute error
    mae = nn.L1Loss()
    mae_loss = mae(Y_pred, Y_test_post).item()

    if verbal:
        print(f"Max difference: {max_diff.item():.2f}% on {max_diff_date:%m/%d}")
        print(f"Correlation: {correlation*100:.2f}%")
        print(f"MAE: {mae_loss:.3f}%")

        # Plot the results
        import matplotlib.pyplot as plt
        plt.plot(Y_test_post.cpu().numpy(), label='True')
        plt.plot(Y_pred.cpu().detach().numpy(), label='Predicted')

        # Fix Y scale
        plt.ylim(0, 60)

        # Label the x-axis with the dates
        days = pd.date_range(start=start_date, end=end_date, freq='D')
        days = [day.strftime("%m-%d") for day in days]
        plt.xticks(range(0, len(days), 30), days[::30], rotation=60)

        # Label the y-axis with definition
        plt.ylabel(y_definition)
        plt.xlabel('Date')

        plt.legend()
        plt.show()

    return correlation, mae_loss

In [ ]:
def train_nn_with_hyperparameters(Qtrain, Qtest, ILITrain, ILITest, input_window_size=7, num_of_features=50, batch_size=28, hidden_size=128, plot_valid_graph=False, random_seed=0, output_window_size = 7):
    torch.manual_seed(random_seed)
    learning_rate = 0.001
    max_epochs = 100
    train_loader, X_train, Y_train, _, _, X_val, Y_val = prepare_data(Qtrain, Qtest, ILITrain, ILITest, input_window_size=input_window_size, output_window_size=output_window_size, num_of_features=num_of_features, batch_size=batch_size, random_seed=0)
    model, val_loss = train_nn(train_loader, X_train, Y_train, X_val, Y_val, hidden_size=hidden_size, learning_rate=learning_rate, max_epochs=max_epochs, input_window_size=input_window_size, output_window_size=output_window_size, plot_graph=plot_valid_graph)
    return model, val_loss

In [ ]:
universal_random_seed = 5

test_seasons_1 = all_seasons[-2:-1]
train_seasons_1 = all_seasons[:-2]
training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]
processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
testing_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_1]

# Hyperparameters
input_window_sizes = [7]
nums_of_features = [60]
hidden_sizes = [128]
batch_sizes = [14]

# Tune hyperparameters
best_hyperparameters_1 = None
best_val_loss_1 = math.inf
for input_window_size in input_window_sizes:
    for num_of_features in nums_of_features:
        for hidden_size in hidden_sizes:
            for batch_size in batch_sizes:
                print("--------------------")
                print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                model, val_loss = train_nn_with_hyperparameters(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                print("Validation loss:", val_loss)
                if val_loss < best_val_loss_1:
                    best_val_loss_1 = val_loss
                    best_hyperparameters_1 = (input_window_size, num_of_features, hidden_size, batch_size)

# Train over entire training data with best hyperparameters
input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = best_hyperparameters_1
print("Best hyperparameters:", best_hyperparameters_1)
model_us_1, val_loss_1 = train_nn_with_hyperparameters(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, num_of_features=num_of_features_1, batch_size=batch_size_1, hidden_size=hidden_size_1, random_seed=universal_random_seed)

train_loader_1, X_train_1, Y_train_1, X_test_1, Y_test_1 = prepare_data_without_validation(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, output_window_size=7, num_of_features=num_of_features_1, batch_size=batch_size_1)

test_seasons_2 = all_seasons[-1:]
train_seasons_2 = all_seasons[:-1]
training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]
processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
testing_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_2]

# Tune hyperparameters
best_hyperparameters_2 = None
best_val_loss_2 = math.inf
for input_window_size in input_window_sizes:
    for num_of_features in nums_of_features:
        for hidden_size in hidden_sizes:
            for batch_size in batch_sizes:
                print("--------------------")
                print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                model, val_loss = train_nn_with_hyperparameters(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                print("Validation loss:", val_loss)
                if val_loss < best_val_loss_2:
                    best_val_loss_2 = val_loss
                    best_hyperparameters_2 = (input_window_size, num_of_features, hidden_size, batch_size)

# Train over entire training data with best hyperparameters
input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = best_hyperparameters_2
print("Best hyperparameters:", best_hyperparameters_2)
model_us_2, val_loss_2 = train_nn_with_hyperparameters(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, num_of_features=num_of_features_2, batch_size=batch_size_2, hidden_size=hidden_size_2, random_seed=universal_random_seed)

train_loader_2, X_train_2, Y_train_2, X_test_2, Y_test_2 = prepare_data_without_validation(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, output_window_size=7, num_of_features=num_of_features_2, batch_size=batch_size_2)

plot_predictions(model_us_1, X_test_1, Y_test_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, y_definition='Number of ILI cases per 100,000 population (US)', savefig=True, filename="./plots/ffnn_us_1_regularized.pdf")
plot_predictions(model_us_2, X_test_2, Y_test_2, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, y_definition='Number of ILI cases per 100,000 population (US)', savefig=True, filename="./plots/ffnn_us_2_regularized.pdf")

# Load England data, baseline: Supervised Learning

In [ ]:
def process_query_data_source_minmax(q_freq_raw, query_ids=query_ids, qmin=None, qmax=None):
    q_freq_raw_filtered  = q_freq_raw[q_freq_raw['Query'].isin(query_ids)]
    q_freq_raw_filtered = q_freq_raw_filtered.pivot(index='Day', columns='Query', values='Frequency')

    # Fill in missing days with NaN
    q_freq_raw_filtered = q_freq_raw_filtered.reindex(range(q_freq_raw_filtered.index.min(), q_freq_raw_filtered.index.max() + 1), fill_value=None)
    # Replace missing values with zeroes
    q_freq_raw_filtered = q_freq_raw_filtered.fillna(0)
    q_freq_raw_filtered_smoothed = q_freq_raw_filtered

    # Apply harmonic mean to the data: for each day, use the harmonic weighted mean of the frequencies back to T-13 (or the start of the data if there are fewer than 14 days of data before T) to smooth the data
    harmonic_length = 7
    harmonic_filter = np.array([(harmonic_length-i)/harmonic_length for i in range(harmonic_length+1)])
    # harmonic_filter = np.array([1/i for i in range(1, harmonic_length+1)])
    harmonic_weight_factor = [1 / sum(harmonic_filter[:i]) for i in range(1, harmonic_length+1)]

    q_freq_raw_filtered_smoothed = deepcopy(q_freq_raw_filtered)
    for i in range(1, q_freq_raw_filtered.shape[0]):
        harmonic_range = min(i + 1, harmonic_length)
        harmonic_values = q_freq_raw_filtered.iloc[i - harmonic_range + 1:i + 1].values
        harmonic_weights = harmonic_filter[:harmonic_range][::-1]
        weighted_sum = (harmonic_values * harmonic_weights[:, None]).sum(axis=0)
        q_freq_raw_filtered_smoothed.iloc[i] = weighted_sum * harmonic_weight_factor[harmonic_range - 1]

    # query_min_target = {}
    # query_max_target = {}
    # for query in q_freq_raw_filtered_smoothed.columns:
    #     query_min_target[query] = q_freq_raw_filtered_smoothed[query].min()
    #     query_max_target[query] = q_freq_raw_filtered_smoothed[query].max()
    
    # Normalize the data
    # for query in q_freq_raw_filtered_smoothed.columns:
    #     if query in qmin and query in qmax:
    #         q_freq_raw_filtered_smoothed[query] = (q_freq_raw_filtered_smoothed[query] - qmin[query]) / (qmax[query] - qmin[query])
    #     else:
    #         q_freq_raw_filtered_smoothed[query] = (q_freq_raw_filtered_smoothed[query] - query_min_target[query]) / (query_max_target[query] - query_min_target[query])

    # Fill NaN values with 0
    q_freq_raw_filtered_smoothed = q_freq_raw_filtered_smoothed.fillna(0)
    return q_freq_raw_filtered_smoothed

In [ ]:
with open('./data/England/queries_filtered.txt') as f:
    raw_england_queries = f.read().splitlines()
query_ids_list_england = [int(r.split('\t')[0]) for r in raw_england_queries]
query_ids_england = set(query_ids_list_england)

In [ ]:
test_seasons = all_seasons[-2:]
train_seasons = all_seasons[:-2]

# Load England data
q_freq_england_raw = pd.read_csv('./data/England/Q_freq.csv')
q_freq_england_smoothed_normalized, qmin_england, qmax_england = process_query_data(q_freq_england_raw, query_ids=query_ids_england)
q_freq_england_smoothed = process_query_data_source_minmax(q_freq_england_raw, query_ids=query_ids_england, qmin=query_min_us, qmax=query_max_us)
ili_raw_england = pd.read_csv('./data/England/ILI_rates_RCGP.csv')
processed_ili_data_england, min_ili_england, max_ili_england = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y")

training_query_data_england = [query_data_for_season(start_date, end_date, q_freq_england_smoothed_normalized) for start_date, end_date in train_seasons]
testing_query_data_england = [query_data_for_season(start_date, end_date, q_freq_england_smoothed_normalized) for start_date, end_date in test_seasons]
training_ili_data_england = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons]
testing_ili_data_england = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons]

In [ ]:
# Plot ILI data by seasons, combined in one plot

plt.plot(pd.concat(training_ili_data_england), label="Training")
plt.plot(pd.concat(testing_ili_data_england), label="Testing")

plt.xlabel("Day")
plt.ylabel("Normalized ILI Rate")

# Change the x-axis to show the dates of start of each season
plt.xticks([day_index(start_date) for start_date, _ in train_seasons + test_seasons], [start_date for start_date, _ in train_seasons + test_seasons], rotation=60)

plt.legend()
plt.tight_layout()
plt.savefig('ILI_Rate_by_Season.pdf', bbox_inches='tight')
plt.show()

In [ ]:
ili_raw_us

In [ ]:
# Plot raw ILI data in Us and England from 2010 to 2020

ploting_us = ili_raw_us[ili_raw_us['Day'].between(day_index('2011-09-01'), day_index('2019-08-31'))]
ploting_england = ili_raw_england[ili_raw_england['Day'].between(day_index('2011-09-01'), day_index('2019-08-31'))]

plt.plot(ploting_us['Day'], ploting_us['ILIRate'], label="US")
plt.plot(ploting_england['Day'], ploting_england['ILIRate'], label="England")

plt.xlabel("Year")
plt.ylabel("ILI Rate")

plt.xticks([day_index(str(year) + '-09-01') for year in range(2011, 2020)], range(2011, 2020), rotation=60)

plt.legend()
plt.tight_layout()

plt.savefig('plots/ILI_Rate_US_England.pdf', bbox_inches='tight')

plt.show()

In [ ]:
q_freq_us_smoothed_englandids, _, _ = process_query_data(q_freq_us_raw, query_ids=query_ids_england)
training_query_data_englandids = [query_data_for_season(start_date, end_date, q_freq_us_smoothed_englandids) for start_date, end_date in train_seasons]

train_seasons = all_seasons[:-1]
training_ili_data_us = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons]

_, qids_correlated_englandids, qids_all_sorted_englandids = filter_correlated_queries(training_query_data_englandids, training_ili_data_us, num_of_features=50)

# Feature mapping, non-uniform weighting

In [ ]:
# Load embeddings of queries
query_embeddings = torch.load('./data/US/queries_embeddings.pt')

def query_similarity(qid1, qid2):
    return F.cosine_similarity(query_embeddings[qid1], query_embeddings[qid2], dim=0).item()

In [ ]:
import numpy as np

# Load embeddings
query_embeddings = torch.load('./data/US/queries_embeddings.pt')

def compute_cosine_similarity_matrix(embeddings):
    embeddings = F.normalize(embeddings, p=2, dim=1)
    similarity_matrix = torch.mm(embeddings, embeddings.t())
    return similarity_matrix

cosine_similarity_matrix = compute_cosine_similarity_matrix(query_embeddings)

def query_mapping_baseline(source_query_ids, target_query_ids, k=1, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed):
    # Does not map, just use the same queries
    feature_dict = {}
    target_query_ids = target_query_ids.intersection(target_query_data.columns)
    target_query_ids = list(target_query_ids)

    for source_query in source_query_ids:
        if source_query in target_query_ids:
            feature_dict[source_query] = [(source_query, 1)]
        else:
            feature_dict[source_query] = []
    return feature_dict

# Define the query mapping function
def query_mapping(source_query_ids, target_query_ids, k=1, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=0.5, uniform_weight=False):
    feature_dict = {}
    
    source_data = source_query_data[source_query_ids].values
    target_query_ids = target_query_ids.intersection(target_query_data.columns)
    target_query_ids = list(target_query_ids)
    target_data = target_query_data[target_query_ids].values
    # Truncate target or source data if the number of days is different
    if source_data.shape[0] < target_data.shape[0]:
        target_data = target_data[:source_data.shape[0]]
    elif source_data.shape[0] > target_data.shape[0]:
        source_data = source_data[:target_data.shape[0]]
    correlation_matrix = np.corrcoef(source_data.T, target_data.T)[:len(source_query_ids), len(source_query_ids):]
    combined_similarity_matrix = semantic_weight * cosine_similarity_matrix[source_query_ids][:, target_query_ids] + (1 - semantic_weight) * torch.tensor(correlation_matrix)
    combined_similarity_matrix = combined_similarity_matrix.numpy()

    for idx, source_query in enumerate(source_query_ids):
        similarities = combined_similarity_matrix[idx]
        top_k_indices = similarities.argsort()[-k:][::-1]
        top_k_similarities = similarities[top_k_indices]
        
        # Normalize the weights using softmax
        if uniform_weight:
            softmax_weights = np.ones(k) / k
        else:
            softmax_weights = np.exp(top_k_similarities) / np.sum(np.exp(top_k_similarities))
        
        # Map the source query to the top K target queries with weights
        feature_dict[source_query] = [(target_query_ids[i], weight) for i, weight in zip(top_k_indices, softmax_weights)]
    
    return feature_dict

# Define the query mapping function
def query_mapping_variable(source_query_ids, target_query_ids, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=0.5, uniform_weight=False, least_k=1):
    feature_dict = {}
    
    source_data = source_query_data[source_query_ids].values
    target_query_ids = target_query_ids.intersection(target_query_data.columns)
    target_query_ids = list(target_query_ids)
    target_data = target_query_data[target_query_ids].values
    # Truncate target or source data if the number of days is different
    if source_data.shape[0] < target_data.shape[0]:
        target_data = target_data[:source_data.shape[0]]
    elif source_data.shape[0] > target_data.shape[0]:
        source_data = source_data[:target_data.shape[0]]
    correlation_matrix = np.corrcoef(source_data.T, target_data.T)[:len(source_query_ids), len(source_query_ids):]
    combined_similarity_matrix = semantic_weight * cosine_similarity_matrix[source_query_ids][:, target_query_ids] + (1 - semantic_weight) * torch.tensor(correlation_matrix)
    combined_similarity_matrix = combined_similarity_matrix.numpy()

    for idx, source_query in enumerate(source_query_ids):
        similarities = combined_similarity_matrix[idx]
        corr = 0
        new_corr = 0
        k = least_k
        while new_corr >= corr and k <= 20:
            corr = new_corr
            top_k_indices = similarities.argsort()[-k:][::-1]
            top_k_similarities = similarities[top_k_indices]
            
            if uniform_weight:
                softmax_weights = np.ones(k) / k
            else:
                softmax_weights = np.exp(top_k_similarities) / np.sum(np.exp(top_k_similarities))
            
            new_corr = np.corrcoef(source_data.T[idx], np.matmul(softmax_weights, target_data.T[top_k_indices]))[0, 1]

            k += 1

        k = k - 1
        top_k_indices = similarities.argsort()[-k:][::-1]
        top_k_similarities = similarities[top_k_indices]

        if uniform_weight:
            softmax_weights = np.ones(k) / k
        else:
            softmax_weights = np.exp(top_k_similarities) / np.sum(np.exp(top_k_similarities))
        
        # Map the source query to the top K target queries with weights
        feature_dict[source_query] = [(target_query_ids[i], weight) for i, weight in zip(top_k_indices, softmax_weights)]
    
    return feature_dict

In [ ]:
# Run model_us on England data with mapped queries
from sklearn.linear_model import LinearRegression

def prepare_data_with_mapping(source_Qtest, Qtest, ILITest, Qtrain, source_Qtrain, ILITrain, mapping, input_window_size=7, output_window_size=7, qids_correlated=qids_correlated, qmin_source=query_min_us, qmax_source=query_max_us):
    # Determine the queries that are correlated with ILI rates based on training data, then filter the testing data based on the correlated queries
    source_Qtest_correlated = [query_data[list(qids_correlated)] for query_data in source_Qtest]
    source_Qtrain_correlated = [query_data[list(qids_correlated)] for query_data in source_Qtrain]

    X_test = deepcopy(source_Qtest_correlated)
    Y_test = deepcopy(ILITest)
    X_train = deepcopy(source_Qtrain_correlated)
    Y_train = deepcopy(ILITrain)

    for original_season, mapped_season in zip(Qtest, X_test):
        for source_query in mapped_season.columns:
            mapped_season[source_query] = 0
            for target_query, weight in mapping[source_query]:
                mapped_season[source_query] += weight * original_season[target_query]

    for original_season, mapped_season in zip(Qtrain, X_train):
        for source_query in mapped_season.columns:
            mapped_season[source_query] = 0
            for target_query, weight in mapping[source_query]:
                mapped_season[source_query] += weight * original_season[target_query]

    # Baseline only: if the query is 0, set it to the average of all other queries
    for season in X_test:
        season_mean = season.mean(axis=1)
        for query in season.columns:
            if season[query].sum() == 0:
                season[query] = season_mean

    for season in X_train:
        season_mean = season.mean(axis=1)
        for query in season.columns:
            if season[query].sum() == 0:
                season[query] = season_mean

    # For each query in qids_correlated, find min and max across X_train and X_test
    qmin, qmax = {}, {}
    for query in qids_correlated:
        qmin[query] = X_train[-1][query].min()

    qmax_all_season = {}
    qmin_all_season = {}
    for query in qids_correlated:
        qmax_all_season[query] = X_train[-1][query].max()
        qmin_all_season[query] = X_train[-1][query].min()

    # Min-max normalize X_test based on source region min and max
    for season in X_test:
        for query in qids_correlated:
            if qmax_all_season[query] - qmin_all_season[query] != 0:
                season[query] = (season[query] - qmin_all_season[query]) / (qmax_all_season[query] - qmin_all_season[query])

    harmonic_length = 7
    harmonic_filter = np.array([(harmonic_length-i)/harmonic_length for i in range(harmonic_length+1)])
    harmonic_weight_factor = [1 / sum(harmonic_filter[:i]) for i in range(1, harmonic_length+1)]
    for k, season in enumerate(X_test):
        season_smoothed = deepcopy(season)
        for i in range(1, season.shape[0]):
            harmonic_range = min(i + 1, harmonic_length)
            harmonic_values = season.iloc[i - harmonic_range + 1:i + 1].values
            harmonic_weights = harmonic_filter[:harmonic_range][::-1]
            weighted_sum = (harmonic_values * harmonic_weights[:, None]).sum(axis=0)
            season_smoothed.iloc[i] = weighted_sum * harmonic_weight_factor[harmonic_range - 1]
        X_test[k] = season_smoothed
        
    qmax_all = 1

    X_test_folds, Y_test_folds = preprocess_fold(X_test, Y_test, input_window_size=input_window_size, output_window_size=output_window_size)
    X_train, Y_train = preprocess_fold(X_train, Y_train, input_window_size=input_window_size, output_window_size=output_window_size)

    # Move data to device
    X_test_folds = X_test_folds.to(device)
    Y_test_folds = Y_test_folds.to(device)
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)

    return X_test_folds, Y_test_folds, X_train, Y_train, X_test, X_train, source_Qtest_correlated, qmax_all, Y_train, Y_test

In [ ]:
def train_nn_transfer(train_loader, X_train, Y_train, X_val, Y_val, hidden_size=128, learning_rate=0.01, max_epochs=200, input_window_size=7, output_window_size=7, plot_graph=False):
    input_size = X_train.shape[1]
    output_size = output_window_size

    print("Input size: ", input_size)

    model = NowcastFCNN(input_size, hidden_size, output_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    patience = 10
    patience_counter = 0
    best_model = None
    best_val_loss = math.inf
    loss_fn = nn.MSELoss()

    warmup_epochs = 5
    val_losses = []

    multiply_counter = 1
        
    # Train the model
    for i in tqdm(range(max_epochs)):
        # Load data in batches
        for X_batch, Y_batch in train_loader:
            model.train()
            optimizer.zero_grad()
            if(X_batch.shape[0] == 1):
                continue
            Y_pred = model(X_batch)
            loss = loss_fn(Y_pred, Y_batch) + 1e-3 * sum(torch.linalg.norm(p, 1) for p in model.parameters())
            loss.backward()
            optimizer.step()
        _, val_loss, _, _ = test_model(model, X_train, Y_train, X_val, Y_val, output_size=output_size, plot_graph=False)
        val_losses.append(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model)
            patience_counter = 0
        elif i > warmup_epochs:
            patience_counter += 1
        if patience_counter >= patience and i > warmup_epochs:
            break

    if plot_graph:
        plt.plot(val_losses)
        plt.show()

    return best_model, best_val_loss

def train_nn_with_hyperparameters_transfer(Qtrain, Qtest, ILITrain, ILITest, input_window_size=7, num_of_features=50, batch_size=28, hidden_size=128, plot_valid_graph=False, random_seed=0):
    torch.manual_seed(random_seed)
    output_window_size = 7
    learning_rate = 0.001
    max_epochs = 100
    train_loader, X_train, Y_train, _, _, X_val, Y_val = prepare_data(Qtrain, Qtest, ILITrain, ILITest, input_window_size=input_window_size, output_window_size=output_window_size, num_of_features=num_of_features, batch_size=batch_size, random_seed=0)
    model, val_loss = train_nn_transfer(train_loader, X_train, Y_train, X_val, Y_val, hidden_size=hidden_size, learning_rate=learning_rate, max_epochs=max_epochs, input_window_size=input_window_size, output_window_size=output_window_size, plot_graph=plot_valid_graph)
    return model, val_loss

In [ ]:
universal_random_seed = 5

test_seasons_1 = all_seasons[-2:-1]
train_seasons_1 = all_seasons[:-2]
training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]
processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
testing_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_1]

# Hyperparameters
input_window_sizes = [7, 14]
nums_of_features = [60, 100]
hidden_sizes = [128, 256]
batch_sizes = [14, 28]

# Tune hyperparameters
best_hyperparameters_1 = None
best_val_loss_1 = math.inf
for input_window_size in input_window_sizes:
    for num_of_features in nums_of_features:
        for hidden_size in hidden_sizes:
            for batch_size in batch_sizes:
                print("--------------------")
                print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                model, val_loss = train_nn_with_hyperparameters(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                print("Validation loss:", val_loss)
                if val_loss < best_val_loss_1:
                    best_val_loss_1 = val_loss
                    best_hyperparameters_1 = (input_window_size, num_of_features, hidden_size, batch_size)

# Train over entire training data with best hyperparameters
input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = best_hyperparameters_1
print("Best hyperparameters:", best_hyperparameters_1)
model_us_1, val_loss_1 = train_nn_with_hyperparameters(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, num_of_features=num_of_features_1, batch_size=batch_size_1, hidden_size=hidden_size_1, random_seed=universal_random_seed)

train_loader_1, X_train_1, Y_train_1, X_test_1, Y_test_1 = prepare_data(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, output_window_size=7, num_of_features=num_of_features_1, batch_size=batch_size_1)

test_seasons_2 = all_seasons[-1:]
train_seasons_2 = all_seasons[:-1]
training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]
processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
testing_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_2]

# Tune hyperparameters
best_hyperparameters_2 = None
best_val_loss_2 = math.inf
for input_window_size in input_window_sizes:
    for num_of_features in nums_of_features:
        for hidden_size in hidden_sizes:
            for batch_size in batch_sizes:
                print("--------------------")
                print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                model, val_loss = train_nn_with_hyperparameters(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                print("Validation loss:", val_loss)
                if val_loss < best_val_loss_2:
                    best_val_loss_2 = val_loss
                    best_hyperparameters_2 = (input_window_size, num_of_features, hidden_size, batch_size)

# Train over entire training data with best hyperparameters
input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = best_hyperparameters_2
print("Best hyperparameters:", best_hyperparameters_2)
model_us_2, val_loss_2 = train_nn_with_hyperparameters(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, num_of_features=num_of_features_2, batch_size=batch_size_2, hidden_size=hidden_size_2, random_seed=universal_random_seed)

train_loader_2, X_train_2, Y_train_2, X_test_2, Y_test_2 = prepare_data(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, output_window_size=7, num_of_features=num_of_features_2, batch_size=batch_size_2)

plot_predictions(model_us_1, X_test_1, Y_test_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, y_definition='Number of ILI cases per 100,000 population (US)')
plot_predictions(model_us_2, X_test_2, Y_test_2, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, y_definition='Number of ILI cases per 100,000 population (US)')

### Two test seasons separated

In [ ]:
universal_random_seed = 50

for universal_random_seed in range(start_seed, end_seed):
    test_seasons_1 = all_seasons[-2:-1]
    train_seasons_1 = all_seasons[:-2]
    training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
    testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]
    processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
    training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
    testing_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_1]

    # Hyperparameters
    input_window_sizes = [7, 14, 21, 28]
    nums_of_features = [60, 70, 80, 90, 100]
    hidden_sizes = [128, 256, 512]
    batch_sizes = [7, 14, 28, 56]

    # Tune hyperparameters
    best_hyperparameters_1 = None
    best_val_loss_1 = math.inf
    for input_window_size in input_window_sizes:
        for num_of_features in nums_of_features:
            for hidden_size in hidden_sizes:
                for batch_size in batch_sizes:
                    print("--------------------")
                    print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                    model, val_loss = train_nn_with_hyperparameters_transfer(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                    print("Validation loss:", val_loss)
                    if val_loss < best_val_loss_1:
                        best_val_loss_1 = val_loss
                        best_hyperparameters_1 = (input_window_size, num_of_features, hidden_size, batch_size)

    # Train over entire training data with best hyperparameters
    input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = best_hyperparameters_1
    print("Best hyperparameters:", best_hyperparameters_1)
    model_us_1, val_loss_1 = train_nn_with_hyperparameters_transfer(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, num_of_features=num_of_features_1, batch_size=batch_size_1, hidden_size=hidden_size_1, random_seed=universal_random_seed)

    train_loader_1, X_train_1, Y_train_1, X_test_1, Y_test_1 = prepare_data_without_validation(training_query_data_us_1, testing_query_data_us_1, training_ili_data_us_1, testing_ili_data_us_1, input_window_size=input_window_size_1, output_window_size=7, num_of_features=num_of_features_1, batch_size=batch_size_1)

    test_seasons_2 = all_seasons[-1:]
    train_seasons_2 = all_seasons[:-1]
    training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
    testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]
    processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
    training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
    testing_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_2]

    # Tune hyperparameters
    best_hyperparameters_2 = None
    best_val_loss_2 = math.inf
    for input_window_size in input_window_sizes:
        for num_of_features in nums_of_features:
            for hidden_size in hidden_sizes:
                for batch_size in batch_sizes:
                    print("--------------------")
                    print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                    model, val_loss = train_nn_with_hyperparameters_transfer(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                    print("Validation loss:", val_loss)
                    if val_loss < best_val_loss_2:
                        best_val_loss_2 = val_loss
                        best_hyperparameters_2 = (input_window_size, num_of_features, hidden_size, batch_size)

    # Train over entire training data with best hyperparameters
    input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = best_hyperparameters_2
    print("Best hyperparameters:", best_hyperparameters_2)
    model_us_2, val_loss_2 = train_nn_with_hyperparameters_transfer(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, num_of_features=num_of_features_2, batch_size=batch_size_2, hidden_size=hidden_size_2, random_seed=universal_random_seed)

    train_loader_2, X_train_2, Y_train_2, X_test_2, Y_test_2 = prepare_data_without_validation(training_query_data_us_2, testing_query_data_us_2, training_ili_data_us_2, testing_ili_data_us_2, input_window_size=input_window_size_2, output_window_size=7, num_of_features=num_of_features_2, batch_size=batch_size_2)

    plot_predictions(model_us_1, X_test_1, Y_test_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, y_definition='Number of ILI cases per 100,000 population (US)')
    plot_predictions(model_us_2, X_test_2, Y_test_2, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, y_definition='Number of ILI cases per 100,000 population (US)')

    # Save the two models, along with the hyperparameters
    torch.save(model_us_1, f'./models/model_us_1_{universal_random_seed}_with_cold.pt')
    torch.save(model_us_2, f'./models/model_us_2_{universal_random_seed}_with_cold.pt')
    with open(f'./models/hyperparameters_us_1_{universal_random_seed}_with_cold.txt', 'w') as f:
        f.write(f"{best_hyperparameters_1}")
    with open(f'./models/hyperparameters_us_2_{universal_random_seed}_with_cold.txt', 'w') as f:
        f.write(f"{best_hyperparameters_2}")

In [ ]:
test_seasons_1 = all_seasons[-2:-1]
train_seasons_1 = all_seasons[:-2]
test_seasons_2 = all_seasons[-1:]
train_seasons_2 = all_seasons[:-1]
training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]
processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
testing_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_1]
training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]
processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
testing_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_2]

In [ ]:
# Load model from local file
def string_to_tuple(s):
    return tuple(map(int, s.strip('()').split(', ')))

def load_model_from_file(universal_random_seed):
    model_us_1 = torch.load(f'./models/model_us_1_{universal_random_seed}_with_cold.pt')
    model_us_2 = torch.load(f'./models/model_us_2_{universal_random_seed}_with_cold.pt')
    with open(f'./models/hyperparameters_us_1_{universal_random_seed}_with_cold.txt', 'r') as f:
        hyperparameters_1 = f.read()
        hyperparameters_1 = string_to_tuple(hyperparameters_1)
    with open(f'./models/hyperparameters_us_2_{universal_random_seed}_with_cold.txt', 'r') as f:
        hyperparameters_2 = f.read()
        hyperparameters_2 = string_to_tuple(hyperparameters_2)
    return model_us_1, model_us_2, hyperparameters_1, hyperparameters_2

In [ ]:
# Test baseline method

query_ids_subset = set(list(qids_all_sorted_englandids)[:500])

corr_season1 = []
corr_season2 = []
mae_season1 = []
mae_season2 = []

train_seasons_1 = all_seasons[:-2]
train_seasons_2 = all_seasons[:-1]
test_seasons_1 = all_seasons[-2:-1]
test_seasons_2 = all_seasons[-1:]

processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2018-08-31')

# Load England data
processed_ili_data_england_1, min_ili_england_1, max_ili_england_1 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2017-08-31', training_start_date='2008-09-01')
training_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_1]
training_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_1]
testing_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_1]
training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]

processed_ili_data_england_2, min_ili_england_2, max_ili_england_2 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2018-08-31', training_start_date='2008-09-01')
training_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_2]
training_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_2]
testing_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_2]
training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]

for seed in tqdm(range(start_seed, end_seed)):
    model_us_1, model_us_2, hyperparameters_1, hyperparameters_2 = load_model_from_file(seed)
    input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = hyperparameters_1
    input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = hyperparameters_2

    qids_correlated_us_1 = filter_correlated_queries(training_query_data_us_1, training_ili_data_us_1, num_of_features=num_of_features_1)[1]

    mapping_us_to_england_1 = query_mapping_baseline(qids_correlated_us_1, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed)

    X_test_mapped_1, Y_test_mapped_1, X_train_mapped_1, Y_train_mapped_1, X_test_1, X_train_1, source_X_test_1, qmax_1, Y_train_1, Y_test_1 = prepare_data_with_mapping(testing_query_data_us_1, testing_query_data_england_1, testing_ili_data_england_1, training_query_data_england_1, training_query_data_us_1, training_ili_data_us_1, mapping_us_to_england_1, input_window_size=input_window_size_1, output_window_size=7, qids_correlated=qids_correlated_us_1)

    qids_correlated_us_2 = filter_correlated_queries(training_query_data_us_2, training_ili_data_us_2, num_of_features=num_of_features_2)[1]

    mapping_us_to_england_2 = query_mapping_baseline(qids_correlated_us_2, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed)

    X_test_mapped_2, Y_test_mapped_2, X_train_mapped_2, Y_train_mapped_2, X_test_2, X_train_2, source_X_test_2, qmax_2, Y_train_2, Y_test_2 = prepare_data_with_mapping(testing_query_data_us_2, testing_query_data_england_2, testing_ili_data_england_2, training_query_data_england_2, training_query_data_us_2, training_ili_data_us_2, mapping_us_to_england_2, input_window_size=input_window_size_2, output_window_size=7, qids_correlated=qids_correlated_us_2)

    corr1, mae1 = plot_predictions_scaled(model_us_1, X_test_mapped_1, Y_test_mapped_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, min_ili_target=min_ili_england_1, max_ili_target=max_ili_england_1, y_definition='Number of ILI cases per 100,000 population (England)')
    corr2, mae2 = plot_predictions_scaled(model_us_2, X_test_mapped_2, Y_test_mapped_2, '2018-09-01', '2019-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, min_ili_target=min_ili_england_2, max_ili_target=max_ili_england_2, y_definition='Number of ILI cases per 100,000 population (England)')

    corr_season1.append(corr1)
    corr_season2.append(corr2)
    mae_season1.append(mae1)
    mae_season2.append(mae2)

In [ ]:
np.mean(corr_season1),np.mean(corr_season2),np.mean(mae_season1),np.mean(mae_season2), (np.mean(corr_season1) + np.mean(corr_season2)) / 2, (np.mean(mae_season1) + np.mean(mae_season2)) / 2

In [ ]:
testing_query_data_us_1[0].mean(axis=1)

In [ ]:
query_ids_subset = set(list(qids_all_sorted_englandids)[:500])
k_value = 2
semantic_weight = 0.2
uniform_weight = False

seed = 58

corr_season1 = []
corr_season2 = []
mae_season1 = []
mae_season2 = []

model_us_1, model_us_2, hyperparameters_1, hyperparameters_2 = load_model_from_file(seed)
input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = hyperparameters_1
input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = hyperparameters_2

train_seasons_1 = all_seasons[:-2]
train_seasons_2 = all_seasons[:-1]
test_seasons_1 = all_seasons[-2:-1]
test_seasons_2 = all_seasons[-1:]

processed_ili_data_us_1, min_ili_1, max_ili_1 = preprocess_ili_data(ili_raw_us, training_end_date='2017-08-31')
processed_ili_data_us_2, min_ili_2, max_ili_2 = preprocess_ili_data(ili_raw_us, training_end_date='2018-08-31')

# Load England data
processed_ili_data_england_1, min_ili_england_1, max_ili_england_1 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2017-08-31', training_start_date='2008-09-01')
training_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_1]
training_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_1]
testing_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_1]
training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]

qids_correlated_us_1 = filter_correlated_queries(training_query_data_us_1, training_ili_data_us_1, num_of_features=num_of_features_1)[1]

# mapping_us_to_england_1 = query_mapping(qids_correlated_us_1, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight)
mapping_us_to_england_1 = query_mapping(qids_correlated_us_1, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight)

X_test_mapped_1, Y_test_mapped_1, X_train_mapped_1, Y_train_mapped_1, X_test_1, X_train_1, source_X_test_1, qmax_1, Y_train_1, Y_test_1 = prepare_data_with_mapping(testing_query_data_us_1, testing_query_data_england_1, testing_ili_data_england_1, training_query_data_england_1, training_query_data_us_1, training_ili_data_us_1, mapping_us_to_england_1, input_window_size=input_window_size_1, output_window_size=7, qids_correlated=qids_correlated_us_1)

# Load England data
processed_ili_data_england_2, min_ili_england_2, max_ili_england_2 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2018-08-31', training_start_date='2008-09-01')
training_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_2]
training_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_2]
testing_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_2]
training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]

qids_correlated_us_2 = filter_correlated_queries(training_query_data_us_2, training_ili_data_us_2, num_of_features=num_of_features_2)[1]

mapping_us_to_england_2 = query_mapping(qids_correlated_us_2, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight)

X_test_mapped_2, Y_test_mapped_2, X_train_mapped_2, Y_train_mapped_2, X_test_2, X_train_2, source_X_test_2, qmax_2, Y_train_2, Y_test_2 = prepare_data_with_mapping(testing_query_data_us_2, testing_query_data_england_2, testing_ili_data_england_2, training_query_data_england_2, training_query_data_us_2, training_ili_data_us_2, mapping_us_to_england_2, input_window_size=input_window_size_2, output_window_size=7, qids_correlated=qids_correlated_us_2)

corr1, mae1 = plot_predictions_scaled(model_us_1, X_test_mapped_1, Y_test_mapped_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, min_ili_target=min_ili_england_1, max_ili_target=max_ili_england_1, y_definition='Number of ILI cases per 100,000 population (England)')
corr2, mae2 = plot_predictions_scaled(model_us_2, X_test_mapped_2, Y_test_mapped_2, '2018-09-01', '2019-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, min_ili_target=min_ili_england_2, max_ili_target=max_ili_england_2, y_definition='Number of ILI cases per 100,000 population (England)')

In [ ]:
# Plot average query data after query mapping, X_test_mapped_1, X_test_mapped_2

plt.plot(X_train_mapped_1.cpu().mean(axis=-1))

# Plot a fitting linear regression line
from sklearn.linear_model import LinearRegression

X = np.arange(len(X_train_mapped_1))
reg = LinearRegression().fit(X.reshape(-1, 1), X_train_mapped_1.cpu().mean(axis=-1).numpy())
plt.plot(X, reg.predict(X.reshape(-1, 1)), color='red', linewidth=2)

# X-axis to start of each season, (use the function day_index)
plt.xticks([day_index(str(year) + '-09-01') - day_index('2008-09-01') for year in range(2008, 2017)], range(2008, 2017), rotation=60)

plt.xlabel('Year')
plt.ylabel('Mean of query frequencies, unnormalized')

plt.tight_layout()

plt.savefig('mean_query_freq.pdf')

In [ ]:
def test_transfer_parameter(k_value=5, semantic_weight=0.2, uniform_weight=False, candidate=500):
    query_ids_subset = set(list(qids_all_sorted_englandids)[:candidate])

    corr_season1 = []
    corr_season2 = []
    mae_season1 = []
    mae_season2 = []

    # Load England data
    _, min_ili_england_1, max_ili_england_1 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2017-08-31', training_start_date='2008-09-01')
    training_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_1]
    testing_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_1]
    testing_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_1]
    training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
    training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
    testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]

    _, min_ili_england_2, max_ili_england_2 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2018-08-31', training_start_date='2008-09-01')
    training_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_2]
    testing_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_2]
    testing_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_2]
    training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
    training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
    testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]

    for seed in tqdm(range(start_seed, end_seed)):
        model_us_1, model_us_2, hyperparameters_1, hyperparameters_2 = load_model_from_file(seed)
        input_window_size_1, num_of_features_1, _, _ = hyperparameters_1
        input_window_size_2, num_of_features_2, _, _ = hyperparameters_2

        qids_correlated_us_1 = filter_correlated_queries(training_query_data_us_1, training_ili_data_us_1, num_of_features=num_of_features_1)[1]

        mapping_us_to_england_1 = query_mapping(qids_correlated_us_1, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight)

        X_test_mapped_1, Y_test_mapped_1, _, _, _, _, _, _, _, _ = prepare_data_with_mapping(testing_query_data_us_1, testing_query_data_england_1, testing_ili_data_england_1, training_query_data_england_1, training_query_data_us_1, training_ili_data_us_1, mapping_us_to_england_1, input_window_size=input_window_size_1, output_window_size=7, qids_correlated=qids_correlated_us_1)

        qids_correlated_us_2 = filter_correlated_queries(training_query_data_us_2, training_ili_data_us_2, num_of_features=num_of_features_2)[1]

        mapping_us_to_england_2 = query_mapping(qids_correlated_us_2, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight)

        X_test_mapped_2, Y_test_mapped_2, _, _, _, _, _, _, _, _ = prepare_data_with_mapping(testing_query_data_us_2, testing_query_data_england_2, testing_ili_data_england_2, training_query_data_england_2, training_query_data_us_2, training_ili_data_us_2, mapping_us_to_england_2, input_window_size=input_window_size_2, output_window_size=7, qids_correlated=qids_correlated_us_2)

        corr1, mae1 = plot_predictions_scaled(model_us_1, X_test_mapped_1, Y_test_mapped_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, min_ili_target=min_ili_england_1, max_ili_target=max_ili_england_1, y_definition='Number of ILI cases per 100,000 population (England)', verbal=False)
        corr2, mae2 = plot_predictions_scaled(model_us_2, X_test_mapped_2, Y_test_mapped_2, '2018-09-01', '2019-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, min_ili_target=min_ili_england_2, max_ili_target=max_ili_england_2, y_definition='Number of ILI cases per 100,000 population (England)', verbal=False)

        corr_season1.append(corr1)
        corr_season2.append(corr2)
        mae_season1.append(mae1)
        mae_season2.append(mae2)

    return corr_season1, corr_season2, mae_season1, mae_season2

In [ ]:
def test_transfer_parameter_variable(semantic_weight=0.2, uniform_weight=False, candidate=500, least_k=1):
    query_ids_subset = set(list(qids_all_sorted_englandids)[:candidate])

    corr_season1 = []
    corr_season2 = []
    mae_season1 = []
    mae_season2 = []

    for seed in tqdm(range(start_seed, end_seed)):
        model_us_1, model_us_2, hyperparameters_1, hyperparameters_2 = load_model_from_file(seed)
        input_window_size_1, num_of_features_1, hidden_size_1, batch_size_1 = hyperparameters_1
        input_window_size_2, num_of_features_2, hidden_size_2, batch_size_2 = hyperparameters_2

        # Load England data
        processed_ili_data_england_1, min_ili_england_1, max_ili_england_1 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2017-08-31', training_start_date='2008-09-01')
        training_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_1]
        testing_query_data_england_1 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_1]
        training_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_1]
        testing_ili_data_england_1 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_1]
        training_ili_data_us_1 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_1]
        training_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_1]
        testing_query_data_us_1 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_1]

        qids_correlated_us_1 = filter_correlated_queries(training_query_data_us_1, training_ili_data_us_1, num_of_features=num_of_features_1)[1]

        mapping_us_to_england_1 = query_mapping_variable(qids_correlated_us_1, query_ids_subset, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight, least_k=least_k)

        X_test_mapped_1, Y_test_mapped_1, X_train_mapped_1, Y_train_mapped_1, X_test_1, X_train_1, source_X_test_1, qmax_1, Y_train_1, Y_test_1 = prepare_data_with_mapping(testing_query_data_us_1, testing_query_data_england_1, testing_ili_data_england_1, training_query_data_england_1, training_query_data_us_1, training_ili_data_us_1, mapping_us_to_england_1, input_window_size=input_window_size_1, output_window_size=7, qids_correlated=qids_correlated_us_1)

        # Load England data
        processed_ili_data_england_2, min_ili_england_2, max_ili_england_2 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2018-08-31', training_start_date='2008-09-01')
        training_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_2]
        testing_query_data_england_2 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_2]
        training_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_2]
        testing_ili_data_england_2 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_2]
        training_ili_data_us_2 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_2]
        training_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_2]
        testing_query_data_us_2 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_2]

        qids_correlated_us_2 = filter_correlated_queries(training_query_data_us_2, training_ili_data_us_2, num_of_features=num_of_features_2)[1]

        mapping_us_to_england_2 = query_mapping_variable(qids_correlated_us_2, query_ids_subset, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight, uniform_weight=uniform_weight, least_k=least_k)

        X_test_mapped_2, Y_test_mapped_2, X_train_mapped_2, Y_train_mapped_2, X_test_2, X_train_2, source_X_test_2, qmax_2, Y_train_2, Y_test_2 = prepare_data_with_mapping(testing_query_data_us_2, testing_query_data_england_2, testing_ili_data_england_2, training_query_data_england_2, training_query_data_us_2, training_ili_data_us_2, mapping_us_to_england_2, input_window_size=input_window_size_2, output_window_size=7, qids_correlated=qids_correlated_us_2)

        corr1, mae1 = plot_predictions_scaled(model_us_1, X_test_mapped_1, Y_test_mapped_1, '2017-09-01', '2018-08-31', output_size=7, min_ili=min_ili_1, max_ili=max_ili_1, min_ili_target=min_ili_england_1, max_ili_target=max_ili_england_1, y_definition='Number of ILI cases per 100,000 population (England)', verbal=False)
        corr2, mae2 = plot_predictions_scaled(model_us_2, X_test_mapped_2, Y_test_mapped_2, '2018-09-01', '2019-08-31', output_size=7, min_ili=min_ili_2, max_ili=max_ili_2, min_ili_target=min_ili_england_2, max_ili_target=max_ili_england_2, y_definition='Number of ILI cases per 100,000 population (England)', verbal=False)

        corr_season1.append(corr1)
        corr_season2.append(corr2)
        mae_season1.append(mae1)
        mae_season2.append(mae2)

    return corr_season1, corr_season2, mae_season1, mae_season2

In [ ]:
performance = {}

for k in [1,2,3,4,5,6,7,8,9,10]:
    for semantic_weight in [0, 0.2, 1.0]:
        corr_season1, corr_season2, mae_season1, mae_season2 = test_transfer_parameter(k_value=k, semantic_weight=semantic_weight, uniform_weight=False, candidate=500)
        performance[(k, semantic_weight)] = np.mean(corr_season1), np.mean(corr_season2), np.mean(mae_season1), np.mean(mae_season2)

In [ ]:
for k in [1,2,3,4,5,6,7,8,9,10]:
    print(f''' \multirow{{3}}{{1em}}{{{k}}} & 0.0 & {performance[(k, 0.0)][0]:.3f} & {performance[(k, 0.0)][2]:.3f} & {performance[(k, 0.0)][1]:.3f} & {performance[(k, 0.0)][3]:.3f} & {(performance[(k, 0.0)][0] + performance[(k, 0.0)][1]) / 2:.3f} & {(performance[(k, 0.0)][2] + performance[(k, 0.0)][3]) / 2:.3f} \\\\ 
 & 0.2 & {performance[(k, 0.2)][0]:.3f} & {performance[(k, 0.2)][2]:.3f} & {performance[(k, 0.2)][1]:.3f} & {performance[(k, 0.2)][3]:.3f} & {(performance[(k, 0.2)][0] + performance[(k, 0.2)][1]) / 2:.3f} & {(performance[(k, 0.2)][2] + performance[(k, 0.2)][3]) / 2:.3f} \\\\ 
 & 1.0 & {performance[(k, 1.0)][0]:.3f} & {performance[(k, 1.0)][2]:.3f} & {performance[(k, 1.0)][1]:.3f} & {performance[(k, 1.0)][3]:.3f} & {(performance[(k, 1.0)][0] + performance[(k, 1.0)][1]) / 2:.3f} & {(performance[(k, 1.0)][2] + performance[(k, 1.0)][3]) / 2:.3f} \\\\ 
 \\hline''')

In [ ]:
results = {}
for semantic_weight in [0, 0.2, 1.0]:
    print(f"semantic_weight: {semantic_weight}")
    corr_season1, corr_season2, mae_season1, mae_season2 = test_transfer_parameter_variable(semantic_weight=semantic_weight, uniform_weight=False, candidate=500, least_k=3)
    results[semantic_weight] = np.mean(corr_season1), np.mean(corr_season2), np.mean(mae_season1), np.mean(mae_season2)

In [ ]:
print(f''' \multirow{{3}}{{1em}}{{Variable}} & 0.0 & {results[0.0][0]:.3f} & {results[0.0][2]:.3f} & {results[0.0][1]:.3f} & {results[0.0][3]:.3f} & {(results[0.0][0] + results[0.0][1]) / 2:.3f} & {(results[0.0][2] + results[0.0][3]) / 2:.3f} \\\\ 
 & 0.2 & {results[0.2][0]:.3f} & {results[0.2][2]:.3f} & {results[0.2][1]:.3f} & {results[0.2][3]:.3f} & {(results[0.2][0] + results[0.2][1]) / 2:.3f} & {(results[0.2][2] + results[0.2][3]) / 2:.3f} \\\\ 
 & 1.0 & {results[1.0][0]:.3f} & {results[1.0][2]:.3f} & {results[1.0][1]:.3f} & {results[1.0][3]:.3f} & {(results[1.0][0] + results[1.0][1]) / 2:.3f} & {(results[1.0][2] + results[1.0][3]) / 2:.3f} \\\\ 
 \\hline''')

In [ ]:
results

In [ ]:
# Format and output results into a file

with open('results/results_transfer_variable.txt', 'w') as f:
    for key, value in results.items():
        f.write(f"{key}: {value[0]*100:.2f}% {value[1]:.2f}\n")

In [ ]:
with open('results/results_transfer_with_cold.txt', 'a') as f:
    for key, value in results.items():
        f.write(f"{key[0]} {key[1]}: {value[0]*100:.2f}% {value[1]:.2f}\n")

In [ ]:
plt.plot(X_train_1.cpu().mean(axis=1))

In [ ]:
ids = [m[0] for m in mapping_us_to_england_1[5276]]

[query_ids_dict[q] for q in ids]

In [ ]:
mapping_us_to_england_1[2040]

In [ ]:
# I want to investigate the spike in early curve for X_test_1_all
X_test_1_all = pd.concat(X_test_1)

# Filter days. Only the first 50 days
X_test_1_all_part = X_test_1_all.iloc[10:110]

# Which query caused the spike?
# Now find the idx of maximum for each query

max_idx = X_test_1_all_part.idxmax()

# Find the 10 earliest maximum of all queries
max_idx.sort_values()[:10]

In [ ]:
# Plot a single query

plt.plot(q_freq_england_smoothed[2040][day_index('2017-09-01'):day_index('2018-08-31')])

In [ ]:
query_ids_dict[2671]

In [ ]:
from matplotlib import pyplot as plt
# Plot X_test_1 and X_test_2 across time (all queries averaged)
X_test_1_all = pd.concat(X_test_1)
X_test_2_all = pd.concat(X_test_2)

plt.plot(X_test_1_all.mean(axis=1), label='X_test_1')
plt.plot(X_test_2_all.mean(axis=1), label='X_test_2')
plt.show()

plt.plot(X_test_1_all.max(axis=1), label='X_test_1')
plt.plot(X_test_2_all.max(axis=1), label='X_test_2')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(X_test_1_all[X_test_1_all.columns[6]], label='X_test_1')
plt.plot(X_test_2_all[X_test_2_all.columns[6]], label='X_test_2')
plt.show()

In [ ]:
universal_random_seed = 55

test_seasons_3 = all_seasons[-3:-2]
train_seasons_3 = all_seasons[:-3]
training_query_data_us_3 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in train_seasons_3]
testing_query_data_us_3 = [query_data_for_season(start_date, end_date, q_freq_us_smoothed) for start_date, end_date in test_seasons_3]
processed_ili_data_us_3, min_ili_3, max_ili_3 = preprocess_ili_data(ili_raw_us, training_end_date='2016-08-31')
training_ili_data_us_3 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in train_seasons_3]
testing_ili_data_us_3 = [ili_data_for_season(start_date, end_date, processed_ili_data) for start_date, end_date in test_seasons_3]

# Hyperparameters
input_window_sizes = [7, 14]
nums_of_features = [60, 100]
hidden_sizes = [128, 256]
batch_sizes = [7, 14, 28]

# Tune hyperparameters
best_hyperparameters_3 = None
best_val_loss_3 = math.inf
for input_window_size in input_window_sizes:
    for num_of_features in nums_of_features:
        for hidden_size in hidden_sizes:
            for batch_size in batch_sizes:
                print("--------------------")
                print("Hyperparameters(Input window size, number of features, hidden size, batch size):", input_window_size, num_of_features, hidden_size, batch_size)
                model, val_loss = train_nn_with_hyperparameters_transfer(training_query_data_us_3, testing_query_data_us_3, training_ili_data_us_3, testing_ili_data_us_3, input_window_size=input_window_size, num_of_features=num_of_features, batch_size=batch_size, hidden_size=hidden_size, random_seed=universal_random_seed, plot_valid_graph=False)
                print("Validation loss:", val_loss)
                if val_loss < best_val_loss_3:
                    best_val_loss_3 = val_loss
                    best_hyperparameters_3 = (input_window_size, num_of_features, hidden_size, batch_size)

# Train over entire training data with best hyperparameters
input_window_size_3, num_of_features_3, hidden_size_3, batch_size_3 = best_hyperparameters_3
print("Best hyperparameters:", best_hyperparameters_3)
model_us_3, val_loss_3 = train_nn_with_hyperparameters_transfer(training_query_data_us_3, testing_query_data_us_3, training_ili_data_us_3, testing_ili_data_us_3, input_window_size=input_window_size_3, num_of_features=num_of_features_3, batch_size=batch_size_3, hidden_size=hidden_size_3, random_seed=universal_random_seed)

train_loader_3, X_train_3, Y_train_3, X_test_3, Y_test_3 = prepare_data_without_validation(training_query_data_us_3, testing_query_data_us_3, training_ili_data_us_3, testing_ili_data_us_3, input_window_size=input_window_size_3, output_window_size=7, num_of_features=num_of_features_3, batch_size=batch_size_3)

plot_predictions(model_us_3, X_test_3, Y_test_3, '2016-09-01', '2017-08-31', output_size=7, min_ili=min_ili_3, max_ili=max_ili_3, y_definition='Number of ILI cases per 100,000 population (US)')

In [ ]:
query_ids_subset = set(list(qids_all_sorted)[:1000])
k_value = 5
semantic_weight = 0.2

# Load England data
processed_ili_data_england_3, min_ili_england_3, max_ili_england_3 = preprocess_ili_data(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2016-08-31', training_start_date='2008-09-01')
training_query_data_england_3 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in train_seasons_3]
testing_query_data_england_3 = [query_data_for_season(start_date, end_date, q_freq_england_smoothed) for start_date, end_date in test_seasons_3]
training_ili_data_england_3 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in train_seasons_3]
testing_ili_data_england_3 = [ili_data_for_season(start_date, end_date, processed_ili_data_england) for start_date, end_date in test_seasons_3]

qids_correlated_us_3 = filter_correlated_queries(training_query_data_us_3, training_ili_data_us_3, num_of_features=num_of_features_3)[1]

mapping_us_to_england_3 = query_mapping(qids_correlated_us_3, query_ids_subset, k=k_value, source_query_data=q_freq_us_smoothed, target_query_data=q_freq_england_smoothed, semantic_weight=semantic_weight)

X_test_mapped_3, Y_test_mapped_3, X_train_mapped_3, Y_train_mapped_3, X_test_3, X_train_3, source_X_test_3, qmax_3, Y_train_3, Y_test_3 = prepare_data_with_mapping(testing_query_data_us_3, testing_query_data_england_3, testing_ili_data_england_3, training_query_data_england_3, training_query_data_us_3, training_ili_data_us_3, mapping_us_to_england_3, input_window_size=input_window_size_3, output_window_size=7, qids_correlated=qids_correlated_us_3)

plot_predictions_scaled(model_us_3, X_test_mapped_3, Y_test_mapped_3, '2016-09-01', '2017-08-31', output_size=7, min_ili=min_ili_3, max_ili=max_ili_3, min_ili_target=min_ili_england_3, max_ili_target=max_ili_england_3, y_definition='Number of ILI cases per 100,000 population (England)')

In [ ]:
from matplotlib import pyplot as plt
X_test_3_all = pd.concat(X_test_3)

# plt.plot(X_test_3_all.max(axis=1), label='X_test_3')
# plt.plot(X_test_3_all.mean(axis=1), label='X_test_3')
for i in range(60):
    plt.plot(X_test_3_all[X_test_3_all.columns[i]], label='X_test_3')
plt.show()

In [ ]:
plt.plot(X_test_1_all.mean(axis=1), label='X_test_2')
for i in range(100):
    plt.plot(X_test_1_all[X_test_1_all.columns[i]], label='X_test_2')
plt.show()

In [ ]:
def preprocess_ili_data_without_normalization(ili_raw, time_format="%Y-%m-%d", training_end_date='2017-08-31', training_start_date='2008-09-01'):
    ili_raw['Day'] = ili_raw['WeekStart'].apply(day_index, time_format=time_format)
    ili_raw = ili_raw.set_index('Day')
    ili_raw = ili_raw[~ili_raw.index.duplicated(keep='first')]
    ili_raw = ili_raw.reindex(range(ili_raw.index.min(), ili_raw.index.max() + 1), fill_value=None)
    ili_raw = ili_raw.reset_index()

    ili_raw = ili_raw[['Day', 'ILIRate']]
    ili_raw = ili_raw[ili_raw['Day'].between(dataset_start_day, dataset_end_day)]

    # Interpolate missing values with linear interpolation
    ili_raw['ILIRate'] = ili_raw['ILIRate'].interpolate(method='linear')

    # Extrapolate so starting days and ending days have values
    ili_raw['ILIRate'] = ili_raw['ILIRate'].bfill()

    ili_raw_training = ili_raw[ili_raw['Day'].between(day_index(training_start_date), day_index(training_end_date))]
    
    return ili_raw['ILIRate'], min_ili, max_ili

In [ ]:
ili_us_raw_without_normalization, min_ili_us, max_ili_us = preprocess_ili_data_without_normalization(ili_raw_us, training_start_date='2008-09-01', training_end_date='2017-08-31')
ili_england_raw_without_normalization, min_ili_england, max_ili_england = preprocess_ili_data_without_normalization(ili_raw_england, time_format="%m/%d/%Y", training_end_date='2017-08-31', training_start_date='2008-09-01')

In [ ]:
# Draw both England and US ILI rates on the same plot, without normalization

plt.plot(list(ili_us_raw_without_normalization[730+365:]), label='US')
plt.plot(list(ili_england_raw_without_normalization[730+365:]), label='England')
plt.legend()

plt.xticks(np.arange(0, len(ili_us_raw_without_normalization[730+365:]), 365), np.arange(2008+3, 2020), rotation=45)
plt.xlabel("Year")
plt.ylabel("ILI Rate")